In [3]:
import pandas as pd
import os
from transformers import AutoTokenizer

path = '../dataset/train/train_no_dup.csv'

In [7]:
''' csv파일을 불러와 subject_entity & object_entity word를 불러와 
    Typed entity marker(with punctuation)처리를 해주는 과정이다. '''


def load_data(path):

    data = pd.read_csv(path)

    sub_entity, sub_type = [], []
    obj_entity, obj_type = [], []
    sub_idx, obj_idx = [], []
    sentence = []

    for i, [x, y, z] in enumerate(zip(data['subject_entity'], data['object_entity'], data['sentence'])):
        sub_typ = x[1:-1].split(':')[-1].split('\'')[-2]
        obj_typ = y[1:-1].split(':')[-1].split('\'')[-2]

        for idx_i in range(len(x)):
            if x[idx_i: idx_i + 9] == 'start_idx':
                sub_start = int(x[idx_i+12:].split(',')[0].strip())
            if x[idx_i: idx_i+7] == 'end_idx':
                sub_end = int(x[idx_i+10:].split(',')[0].strip())

            if y[idx_i: idx_i + 9] == 'start_idx':
                obj_start = int(y[idx_i+12:].split(',')[0].strip())
            if y[idx_i: idx_i+7] == 'end_idx':
                obj_end = int(y[idx_i+10:].split(',')[0].strip())

        sub_i = [sub_start, sub_end]
        obj_i = [obj_start, obj_end]

        sub_entity.append(z[sub_i[0]: sub_i[1]+1])
        obj_entity.append(z[obj_i[0]: obj_i[1]+1])
        sub_type.append(sub_typ)
        sub_idx.append(sub_i)
        obj_type.append(obj_typ)
        obj_idx.append(obj_i)

        if sub_i[0] < obj_i[0]:
            z = z[:sub_i[0]] + '@*'+sub_typ+'*' + \
                z[sub_i[0]: sub_i[1]+1] + '@' + z[sub_i[1]+1:]
            z = z[:obj_i[0]+7] + '#^' + obj_typ + '^' + \
                z[obj_i[0]+7: obj_i[1]+8] + '#' + z[obj_i[1]+8:]
        else:
            z = z[:obj_i[0]] + '#^' + obj_typ + '^' + \
                z[obj_i[0]: obj_i[1]+1] + '#' + z[obj_i[1]+1:]
            z = z[:sub_i[0]+7] + '@*'+sub_typ+'*' + \
                z[sub_i[0]+7: sub_i[1]+8] + '@' + z[sub_i[1]+8:]

        sentence.append(z)

    df = pd.DataFrame({'id': data['id'], 'sentence': sentence, 'subject_entity': sub_entity, 'object_entity': obj_entity,
                      'subject_type': sub_type, 'object_type': obj_type, 'label': data['label'],
                       'subject_idx': sub_idx, 'object_idx': obj_idx})

    return df

In [8]:
''' Typed entity marker(with punctuation)처리된 data를 token화 하고 concat '''


def tokenized_dataset(dataset, tokenizer):
    # tokenizer에 따라 sentence를 tokenizing

    concat_entity = []
    for e01, e02, t01, t02 in zip(dataset['subject_entity'], dataset['object_entity'], dataset['subject_type'], dataset['object_type']):
        temp = '@*' + t01 + '*' + e01 + '@' + '와' + \
            '#^' + t02 + '^' + e02 + '#' + '의 관계'
        concat_entity.append(temp)

    tokenized_sentences = tokenizer(
        concat_entity,
        list(dataset['sentence']),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
        add_special_tokens=True,
        return_token_type_ids=True
    )

    return tokenized_sentences

In [9]:
train_df = load_data(path)
train_df

,id,sentence,subject_entity,object_entity,subject_type,object_type,label,subject_idx,object_idx
0,0,〈Something〉는 #^PER^조지 해리슨#이 쓰고 @*ORG*비틀즈@가 196...,비틀즈,조지 해리슨,ORG,PER,no_relation,"[24, 26]","[13, 18]"
1,1,호남이 기반인 바른미래당·#^ORG^대안신당#·@*ORG*민주평화당@이 우여곡절 끝...,민주평화당,대안신당,ORG,ORG,no_relation,"[19, 23]","[14, 17]"
2,2,K리그2에서 성적 1위를 달리고 있는 @*ORG*광주FC@는 지난 26일 #^ORG...,광주FC,한국프로축구연맹,ORG,ORG,org:member_of,"[21, 24]","[34, 41]"
3,3,균일가 생활용품점 (주)@*ORG*아성다이소@(대표 #^PER^박정부#)는 코로나1...,아성다이소,박정부,ORG,PER,org:top_members/employees,"[13, 17]","[22, 24]"
4,4,#^DAT^1967#년 프로 야구 드래프트 1순위로 @*ORG*요미우리 자이언츠@에...,요미우리 자이언츠,1967,ORG,DAT,no_relation,"[22, 30]","[0, 3]"
...,...,...,...,...,...,...,...,...,...
32423,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,유기준,부산 서구·동구,PER,LOC,per:employee_of,"[93, 95]","[100, 107]"
32424,32466,"법포는 다시 @*PER*최시형@, 서병학, #^PER^손병희# 직계인 북접과 다시 ...",최시형,손병희,PER,PER,per:colleagues,"[7, 9]","[17, 19]"
32425,32467,@*ORG*완도군@(군수 #^PER^신우철#)이 국토교통부에서 실시한 '2019 교...,완도군,신우철,ORG,PER,org:top_members/employees,"[0, 2]","[7, 9]"
32426,32468,"중앙일보, @*ORG*JTBC@ 회장을 지낸 이후 #^ORG^중앙홀딩스# 회장, 재...",JTBC,중앙홀딩스,ORG,ORG,no_relation,"[6, 9]","[21, 25]"


In [10]:
''' 토큰화 후 첫 index로 잘 되었는지 확인한다.
패딩된 token 확인, ids 확인, attention_mask 확인, type_ids 확인 '''

MODEL_NAME = "klue/roberta-small"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenized_train = tokenized_dataset(train_df, tokenizer)

print(tokenized_train[0].tokens)
print('='*50)
print(tokenized_train[0].ids)
print('='*50)
print(tokenized_train[0].attention_mask)
print('='*50)
print(tokenized_train[0].type_ids)
print('='*50)

['[CLS]', '@', '*', 'O', '##R', '##G', '*', '비틀즈', '@', '와', '#', '^', 'PER', '^', '조지', '해리', '##슨', '#', '의', '관계', '[SEP]', '〈', 'So', '##me', '##th', '##ing', '〉', '는', '#', '^', 'PER', '^', '조지', '해리', '##슨', '#', '이', '쓰', '##고', '@', '*', 'O', '##R', '##G', '*', '비틀즈', '@', '가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [ ]:
''' 위의 과정을 본 코드에서는 train.py에서 처리 해 주는데 load_data.py에서 불러와
train_dataset = load_data("../dataset/train/train_no_dup.csv")로 간단하게 처리한 후
tokenized_train = tokenized_dataset(augmented_train_dataset, tokenizer)로 사용한다. '''